In [ ]:
!pip install transformers
!pip install sent2vec
!pip install bleu

     |████████████████████████████████| 2.2MB 13.1MB/s 
     |████████████████████████████████| 3.3MB 14.8MB/s 
     |████████████████████████████████| 870kB 52.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=8d0b0feac72a739cb831cf6e31243d5bab4c5c9deae3439c4251cbe772999932
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
  Created wheel for bleu: filename=bleu-0.3-cp37-none-any.whl size=5802 sha256=3fb83c1ddcc85f69eea85f36ed620770f50ab00ef81ba60e8a2e9173ab58d3b3
  Stored in directory: /root/.cache/pip/wheels/a9/95/e7/cb43a1c509c38fedbee6223963e34a51a94d8991f3b3e1888e
  Created wheel for efficiency: filename=efficiency-0.5-cp37-none-any.whl size=20285 sha256=a97b92322606e44e7890c44258b6ffb748388c802c5b56001c195db235fc1500
  Stored in directory: /root/.cache/pip/wheels/94/fd/e6/2d34e80768fb2d001fc35b033a48114b1afdb6ea020b4cb2c7
Successfully built 

In [ ]:
!pip install --upgrade nltk
import nltk
nltk.download('wordnet')


     |████████████████████████████████| 1.4MB 12.1MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp37-none-any.whl size=1434676 sha256=c49a0ae1ec7d7a0d6b24e6157d2013078a3e005ce8f8d3b9a496e966c51dd411
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from scipy import spatial
from sent2vec.vectorizer import Vectorizer
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering, Trainer, TrainingArguments, EarlyStoppingCallback, TrainerCallback
import torch
import json
from pathlib import Path
import collections
import numpy as np
import matplotlib.pyplot as plt
from bleu import list_bleu,multi_list_bleu ##import bleu metric
from nltk.translate import meteor_score ## import meteor metric

In [ ]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/gdrive', force_remount=True)


Mounted at /gdrive


In [ ]:
path = "/gdrive/MyDrive/ECE692:Project3Paper/"

In [ ]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    question_ids = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                question_id = qa['id']
                question_ids.append(question_id)
                if qa['answers'] == []:
                    answers.append({'text': "", 'answer_start': -1, 'answer_end': -1})
                    contexts.append(context)
                    questions.append(question)
                    #question_ids.append(question_id)
                else:
                    for answer in qa['answers']:
                        answers.append(answer)
                        contexts.append(context)
                        questions.append(question)
                        #question_ids.append(question_id)

    return contexts, questions, question_ids, answers

In [ ]:
def create_question_answers_dict(questions, answers):

    q_index = 0
    q_a_dict = {}
    quest = ''
    inner_dict = {}
    ans_list = []
    for i, (q, a) in enumerate(zip(questions, answers)):
        #print('q', q, 'quest', quest)
        # new question - append data
        if q != quest:

            if len(ans_list) > 0:
                inner_dict['answers'] = ans_list
                q_a_dict[q_index] = inner_dict
                q_index = i
                inner_dict = {}
                # q_index += 1
            ans_list = []
            inner_dict['question'] = q
            quest = q
            ans_list.append(a['text'])
        elif q == quest:
            ans_list.append(a['text'])
            quest = q
        # bug fix
        if i == len(questions) - 1:
            inner_dict['question'] = q
            if len(ans_list) > 0:
                inner_dict['answers'] = ans_list
                q_a_dict[i-len(ans_list)+1] = inner_dict
    print(q_a_dict)
    return q_a_dict

In [ ]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        if answer['answer_start'] == -1:
          continue

        gold_text = answer['text'].lower()
        start_idx = answer['answer_start']
        print(context)
        print(start_idx, len(gold_text))
        print(gold_text)
        end_idx = start_idx + len(gold_text)
        
        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx].lower() == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1].lower() == gold_text:
           
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2].lower() == gold_text:
           
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

        else:
            print('END INDEX NOT FOUND (found', context[start_idx:end_idx] , ')')

In [ ]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        if answers[i]['answer_start'] == -1:
            start_positions.append(tokenizer.model_max_length)
            end_positions.append(tokenizer.model_max_length)
            continue
        # print(answers[i])
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))
        # if None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

In [ ]:
class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

In [ ]:
training_path = path + 'json/final_version/5/christie_training.json'
validation_path = path + 'json/final_version/5/christie_validation.json'
testing_path = path + 'json/final_version/5/christie_test.json'

In [ ]:
train_contexts, train_questions, train_questions_ids, train_answers = read_squad(training_path)
val_contexts, val_questions, val_questions_ids, val_answers = read_squad(validation_path)
test_contexts, test_questions, test_questions_ids, test_answers = read_squad(testing_path)
train_dict = create_question_answers_dict(train_questions, train_answers)
val_dict = create_question_answers_dict(val_questions, val_answers)
test_dict = create_question_answers_dict(test_questions, test_answers)

print(train_contexts)
print(train_questions)
print(train_answers)
add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)
add_end_idx(test_answers, test_contexts)

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased-distilled-squad')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)
test_encodings = tokenizer(test_contexts, test_questions, truncation=True, padding=True)

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)
add_token_positions(test_encodings, test_answers)

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)
test_dataset = SquadDataset(test_encodings)

model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad")

# Test on validation set
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')

{0: {'question': 'The man asked to see who?', 'answers': ['Mr. Pace']}, 1: {'question': 'Mr. Pace seemed what?', 'answers': ['puzzled']}, 2: {'question': 'inside was Mr. Pace, what happened?', 'answers': ['Mr. Pace, all shot and bleeding']}, 3: {'question': 'Who called to see Mr. Pace?', 'answers': ['He had a black beard']}, 4: {'question': 'Where is the husband?', 'answers': ['already left for London']}, 5: {'question': 'Was Mr. Pace his usual self?', 'answers': ['Most normal in every way']}, 6: {'question': "How large was Mr. Pace's fortune?", 'answers': ['']}, 7: {'question': 'Who is an exceptionally fine linguist?', 'answers': ['Captain Daniels', 'Captain', 'Daniels']}, 10: {'question': 'Who has served in the English Army?', 'answers': ['Captain Daniels', 'Captain', 'Daniels']}, 13: {'question': 'Who is an extremely able secretary?', 'answers': ['Captain Daniels', 'Captain', 'Daniels']}, 16: {'question': 'Who is of no particular family?', 'answers': ['Captain Daniels', 'Captain', '

In [ ]:
# this method computes string comparison accuracy and cosine distance accuracy
def check_accuracy(actual_answers, predicted_answer):
    cosine_val = 0
    lexical_val = 0

    for actual_answer in actual_answers:
        if predicted_answer == '' and actual_answer != '' or predicted_answer != '' and actual_answer == '':
            continue

        vectorizer = Vectorizer()
        vectorizer.bert([predicted_answer, actual_answer.lower()])
        vectors_bert = vectorizer.vectors
        dist_answer = spatial.distance.cosine(vectors_bert[0], vectors_bert[1])

        if dist_answer < 0.03:
            cosine_val = 1
        if (predicted_answer.lower() in actual_answer.lower()) or (actual_answer.lower() in predicted_answer.lower()):
            lexical_val = 1
    # print('Lex val %d, cosine val:%d'%(lexical_val, cosine_val))
    return cosine_val, lexical_val

In [ ]:

"""Official evaluation script for SQuAD version 2.0.

In addition to basic functionality, we also compute additional statistics and
plot precision-recall curves if an additional na_prob.json file is provided.
This file is expected to map question ID's to the model's predicted probability
that a question is unanswerable.
"""
import argparse
import collections
import json
import numpy as np
import os
import re
import string
import sys


def make_qid_to_has_ans(dataset):
  qid_to_has_ans = {}
  for article in dataset:
    for p in article['paragraphs']:
      for qa in p['qas']:
        qid_to_has_ans[qa['id']] = bool(qa['answers'])
  return qid_to_has_ans

def normalize_answer(s):
  """Lower text and remove punctuation, articles and extra whitespace."""
  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)
  def white_space_fix(text):
    return ' '.join(text.split())
  def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)
  def lower(text):
    return text.lower()
  return white_space_fix(remove_articles(remove_punc(lower(s))))

def get_tokens(s):
  if not s: return []
  return normalize_answer(s).split()

def compute_exact(a_gold, a_pred):
  return int(normalize_answer(a_gold) == normalize_answer(a_pred))

def compute_f1(a_gold, a_pred):
  gold_toks = get_tokens(a_gold)
  pred_toks = get_tokens(a_pred)
  common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
  num_same = sum(common.values())
  if len(gold_toks) == 0 or len(pred_toks) == 0:
    # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
    return int(gold_toks == pred_toks)
  if num_same == 0:
    return 0
  precision = 1.0 * num_same / len(pred_toks)
  recall = 1.0 * num_same / len(gold_toks)
  f1 = (2 * precision * recall) / (precision + recall)
  return f1

def get_raw_scores(dataset, preds):
  exact_scores = {}
  f1_scores = {}
  for article in dataset:
    for p in article['paragraphs']:
      for qa in p['qas']:
        qid = qa['id']
        gold_answers = [a['text'] for a in qa['answers']
                        if normalize_answer(a['text'])]
        if not gold_answers:
          # For unanswerable questions, only correct answer is empty string
          gold_answers = ['']
        if qid not in preds:
          print('Missing prediction for %s' % qid)
          continue
        a_pred = preds[qid]
        # Take max over all gold answers
        exact_scores[qid] = max(compute_exact(a, a_pred) for a in gold_answers)
        f1_scores[qid] = max(compute_f1(a, a_pred) for a in gold_answers)
  return exact_scores, f1_scores

def apply_no_ans_threshold(scores, na_probs, qid_to_has_ans, na_prob_thresh):
  new_scores = {}
  for qid, s in scores.items():
    pred_na = na_probs[qid] > na_prob_thresh
    if pred_na:
      new_scores[qid] = float(not qid_to_has_ans[qid])
    else:
      new_scores[qid] = s
  return new_scores

def calculate_squad_em_f1(dataset_path, questions_ids, preds_list):
  with open(dataset_path) as f:
    dataset_json = json.load(f)
    dataset = dataset_json['data']
  preds = {}
  for i in range(0, len(preds_list)):
    preds[questions_ids[i]] = preds_list[i]
  na_probs = {k: 0.0 for k in preds.keys()}
  qid_to_has_ans = make_qid_to_has_ans(dataset)
  has_ans_qids = [k for k, v in qid_to_has_ans.items() if v]
  no_ans_qids = [k for k, v in qid_to_has_ans.items() if not v]
  exact_raw, f1_raw = get_raw_scores(dataset, preds)
  exact_thresh = apply_no_ans_threshold(exact_raw, na_probs, qid_to_has_ans, 1.0)
  f1_thresh = apply_no_ans_threshold(f1_raw, na_probs, qid_to_has_ans, 1.0)
  mean_em = np.mean([v for v in exact_thresh.values()])
  mean_f1 = np.mean([v for v in f1_thresh.values()])

  return mean_em, mean_f1




In [ ]:
# method to calculate Jacard similiarity score
def get_jaccard_sim(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
def compute_bleu_score(correct_answers, predicted_answer):
  bleu_score = 0
  for corr_answer in correct_answers:
    if corr_answer.strip() == '' and predicted_answer.strip() == '':
      bleu_score = 1
    else:  
      bleu_score = list_bleu([corr_answer.lower()], [predicted_answer.lower()])
    # print("Blue score %f"% bleu_score)
    if bleu_score > 70.0:
      return 1
  return 0    


In [ ]:
def compute_meteor_score(correct_answers, predicted_answer):
  met_score = 0
  for corr_answer in correct_answers:
    if corr_answer.strip() == '' and predicted_answer.strip() == '':
      return 1
    else:
      met_score = meteor_score.meteor_score([corr_answer.lower()], predicted_answer.lower())
      if met_score > 0.7:
        return 1
  return 0  

In [ ]:
def compute_f1_score(correct_answers, predicted_answer):
  f1_sc = 0
  for corr_answer in correct_answers:
    if corr_answer.strip() == '' and predicted_answer.strip() == '':
      return 1
    else:
      f1_sc = compute_f1(corr_answer.lower(), predicted_answer.lower())
      if f1_sc > 0.7:
        return 1
  return 0 

In [ ]:
def compute_em_score(correct_answers, predicted_answer):
  em_sc = 0
  for corr_answer in correct_answers:
    if corr_answer.strip() == '' and predicted_answer.strip() == '':
      return 1
    else:
      em_sc = compute_exact(corr_answer.lower(), predicted_answer.lower())
      if em_sc > 0.9:
        return 1
  return 0

In [ ]:
def compute_jacard_score(correct_answers, predicted_answer):
  jc_sc = 0
  for corr_answer in correct_answers:
    if corr_answer.strip() == '' and predicted_answer.strip() == '':
      return 1
    else:
      jc_sc = get_jaccard_sim(corr_answer.lower(), predicted_answer.lower())
      if jc_sc > 0.7:
        return 1
  return 0

In [ ]:
for i in range(0, len(test_questions)):
  print(test_questions[i], test_answers[i])

print(test_dict.values())


print(list(test_dict.keys()))

for key in test_dict.keys():
  print(key)

for val in test_dict.values():
  print(val)

Whose promptness in restarting the car saved the Premier's life? {'text': 'O’Murphy', 'answer_start': 87, 'answer_end': 95}
Whose conduct in leaving the main road was suspicious? {'text': 'O’Murphy', 'answer_start': 87, 'answer_end': 95}
Who disappeared in a highly suggestive manner? {'text': 'O’Murphy', 'answer_start': 87, 'answer_end': 95}
Who is a Scotland Yard man? {'text': 'O’Murphy', 'answer_start': 87, 'answer_end': 95}
What did O'Murphy restart? {'text': 'car', 'answer_start': 310, 'answer_end': 313}
What did O'Murphy restart? {'text': 'the car', 'answer_start': 306, 'answer_end': 313}
Who was found gagged, bound, and chloroformed? {'text': 'Daniels', 'answer_start': 448, 'answer_end': 455}
Who speaks too many languages for a good Englishman? {'text': 'Daniels', 'answer_start': 448, 'answer_end': 455}
When is allied conference to be held {'text': 'to-morrow—Thursday—evening', 'answer_start': 231, 'answer_end': 257}
Where is allied conference to be held {'text': 'Versailles', 'a

In [ ]:
print(test_dict)
dict_keys = list(test_dict.keys())
print(dict_keys)
print(dict_keys[-1])

{0: {'question': "Whose promptness in restarting the car saved the Premier's life?", 'answers': ['O’Murphy']}, 1: {'question': 'Whose conduct in leaving the main road was suspicious?', 'answers': ['O’Murphy']}, 2: {'question': 'Who disappeared in a highly suggestive manner?', 'answers': ['O’Murphy']}, 3: {'question': 'Who is a Scotland Yard man?', 'answers': ['O’Murphy']}, 4: {'question': "What did O'Murphy restart?", 'answers': ['car', 'the car']}, 6: {'question': 'Who was found gagged, bound, and chloroformed?', 'answers': ['Daniels']}, 7: {'question': 'Who speaks too many languages for a good Englishman?', 'answers': ['Daniels']}, 8: {'question': 'When is allied conference to be held', 'answers': ['to-morrow—Thursday—evening']}, 9: {'question': 'Where is allied conference to be held', 'answers': ['Versailles']}, 10: {'question': 'Who started and maintained the Pacifist propaganda', 'answers': ['the German agents']}, 11: {'question': 'Where did the abduction take place', 'answers': [

In [ ]:
CUDA_LAUNCH_BLOCKING=1
questions_count = len(test_questions)
model.to(device)
correct = 0
correct_cosine = 0
dict_index = 0
f1_scores = []
em_scores = []
bleu_scores = []
meteor_scores = []
jc_scores = []
predictions_list = []
dict_keys = list(test_dict.keys())
print(dict_keys)
for i in range(len(test_questions)):
    context = test_contexts[i]
    question = test_questions[i]
    answer = test_answers[i]

    encoding = tokenizer.encode_plus(question, context, max_length=1024)
    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]
    start_scores, end_scores = model(torch.tensor([input_ids], device=device),
                                     attention_mask=torch.tensor([attention_mask],
                                                                 device=device), return_dict=False)
    ans_tokens = input_ids[torch.argmax(start_scores.cpu()): torch.argmax(end_scores.cpu()) + 1]
    answer_tokens = tokenizer.convert_ids_to_tokens(ans_tokens, skip_special_tokens=True)
    all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    print(question)
    answer_tokens_to_string = tokenizer.convert_tokens_to_string(answer_tokens)
    # print('Predicted: %s Actual: %s'%(answer_tokens_to_string, answer['text']))

    print(i)
    if i in dict_keys:
      dict_index = i
    # actual_answers = answer['text'].lower() 
    print("%d %d"%(i, dict_index))
    actual_answers = test_dict[dict_index]['answers']
    predicted_answers = answer_tokens_to_string.lower()
    print('Actual: %s Predicted: %s'%(actual_answers, predicted_answers))
    cosine_value, lexical_value = check_accuracy(actual_answers, predicted_answers)
    correct_cosine += cosine_value
    correct += lexical_value
    f1 = (compute_f1_score(actual_answers, predicted_answers))
    # print("F1 score: %f "% f1)
    f1_scores.append(f1)
    em  = compute_em_score(actual_answers, predicted_answers)
    # print("EM score: %f "% em)
    em_scores.append(em)
    bleu = compute_bleu_score(actual_answers, predicted_answers)
    # print("Bleu score: %f "% bleu)
    bleu_scores.append(bleu)
    met = compute_meteor_score(actual_answers, predicted_answers)
    # print("Meteor score: %f "% met)
    meteor_scores.append(met)
    jc = compute_jacard_score(actual_answers, predicted_answers)
    # print("Jacard similarity score: %f "% jc)
    jc_scores.append(jc)

    # if i+1 == len(val_questions) or val_questions[i+1] !=  val_questions[i]:
    #  predictions_list.append(predicted_answers)

    if i+1 == len(test_questions) or test_questions[i+1] !=  test_questions[i]:
      predictions_list.append(predicted_answers)  

#mean_em, mean_f1 = calculate_squad_em_f1(validation_path, val_questions_ids, 
#                                         predictions_list)
mean_em, mean_f1 = calculate_squad_em_f1(testing_path, test_questions_ids, 
                                         predictions_list)


print("Mean string comparison accuracy: {}".format(correct / questions_count))
print("Mean Cosine distance accuracy: {}".format(correct_cosine / questions_count))
print("Mean F1 Score: {}".format(np.mean(f1_scores)))
print("Mean EM Score: {}".format(np.mean(em_scores)))
print("Official SQuAD Mean F1 Score: {}".format(mean_f1))
print("Official SQuAD Mean EM Score: {}".format(mean_em))
print("Mean Bleu Score: {}".format(np.mean(bleu_scores)))
print("Mean Meteor Score: {}".format(np.mean(meteor_scores)))
print("Mean Jacard Similarity Score: {}".format(np.mean(jc_scores)))
print()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 14, 15, 17, 19, 21, 22, 23]
Whose promptness in restarting the car saved the Premier's life?
0
0 0
Actual: ['O’Murphy'] Predicted: daniels



Whose conduct in leaving the main road was suspicious?
1
1 1
Actual: ['O’Murphy'] Predicted: o ’ murphy
Who disappeared in a highly suggestive manner?
2
2 2
Actual: ['O’Murphy'] Predicted: he is an irishman from county clare
Who is a Scotland Yard man?
3
3 3
Actual: ['O’Murphy'] Predicted: daniels
What did O'Murphy restart?
4
4 4
Actual: ['car', 'the car'] Predicted: the car
What did O'Murphy restart?
5
5 4
Actual: ['car', 'the car'] Predicted: the car
Who was found gagged, bound, and chloroformed?
6
6 6
Actual: ['Daniels'] Predicted: does not look as though he had anything to do with the matter
Who speaks too many languages for a good Englishman?
7
7 7
Actual: ['Daniels'] Predicted: daniels


RuntimeError: ignored

In [ ]:
def plot_ft_loss(val_loss):
  x = [i for i in range(0, len(val_loss))]
  plt.plot(x, val_loss, label='Validation')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.title('Fine-tuning Loss')
  plt.show()
  plt.clf()

In [ ]:
# Fine tuning Option A
# Option A
model = DistilBertForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad")

# best results so far per device train/eval batch size = 16, epochs = 2

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# device = torch.device('cpu')
model.to(device)
training_args = TrainingArguments(
    output_dir=path + '/results',  # output directory
    num_train_epochs=400,  # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=64,  # batch size for evaluation
    warmup_steps=50,  # number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # strength of weight decay
    logging_dir='./logs',  # directory for storing logs
    logging_steps=1,
    learning_rate=1e-5,
    load_best_model_at_end = True,
    evaluation_strategy='epoch'

)

val_loss = []
class LossCallback(TrainerCallback):
  def on_evaluate(self, args, state, control, metrics, **kwargs):
    val_loss.append(metrics['eval_loss'])

trainer = Trainer(
    model=model,  # the instantiated hugging face Transformers model to be trained
    args=training_args,  # training arguments, defined above
    train_dataset=train_dataset,  # training dataset
    eval_dataset=val_dataset,  # evaluation dataset
    callbacks=[EarlyStoppingCallback(early_stopping_patience = 4), LossCallback()]
)

trainer.train()

plot_ft_loss(val_loss)

# Test on validation set
questions_count = len(test_questions)

correct = 0
correct_cosine = 0
correct_bleu = 0
correct_meteor = 0
correct_f1 = 0
correct_em = 0
dict_index = 0
f1_scores = []
em_scores = []
bleu_scores = []
meteor_scores = []
jc_scores = []
predictions_list = []

for i in range(len(test_questions)):
    context = test_contexts[i]
    question = test_questions[i]
    answer = test_answers[i]

    encoding = tokenizer.encode_plus(question, context, max_length=1024)
    input_ids, attention_mask = encoding["input_ids"], encoding["attention_mask"]
    start_scores, end_scores = model(torch.tensor([input_ids], device=device),
                                     attention_mask=torch.tensor([attention_mask], device=device), return_dict=False)
    ans_tokens = input_ids[torch.argmax(start_scores.cpu()): torch.argmax(end_scores.cpu()) + 1]
    answer_tokens = tokenizer.convert_ids_to_tokens(ans_tokens, skip_special_tokens=True)
    all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer_tokens_to_string = tokenizer.convert_tokens_to_string(answer_tokens)
    print(question)
    # bug fix
    '''print(test_dict.keys())
    dict_keys = list(test_dict)
    if key == dict_keys[-1] and i >= key:
      dict_index = dict_keys[index]
      break
    elif  key > i :
      dict_index = dict_keys[index-1]
      break'''

    '''for index, key in enumerate(dict_keys):
      
      if key == dict_keys[-1] and i >= key:
        dict_index = dict_keys[index]
        break
      elif  key > i :
        dict_index = dict_keys[index-1]
        break'''

    print(i)
    if i in dict_keys:
      dict_index = i
    # actual_answers = answer['text'].lower() 
    actual_answers = test_dict[dict_index]['answers']
    
    predicted_answers = answer_tokens_to_string.lower()
    print('Actual: %s Predicted: %s' % (actual_answers, predicted_answers))
    cosine_value, lexical_value = check_accuracy(actual_answers, predicted_answers)
    correct_cosine += cosine_value
    correct += lexical_value
    f1 = (compute_f1_score(actual_answers, predicted_answers))
    # print("F1 score: %f "% f1)
    f1_scores.append(f1)
    em  = compute_em_score(actual_answers, predicted_answers)
    # print("EM score: %f "% em)
    em_scores.append(em)
    bleu = compute_bleu_score(actual_answers, predicted_answers)
    # print("Bleu score: %f "% bleu)
    bleu_scores.append(bleu)
    met = compute_meteor_score(actual_answers, predicted_answers)
    # print("Meteor score: %f "% met)
    meteor_scores.append(met)
    jc = compute_jacard_score(actual_answers, predicted_answers)
    # print("Jacard similarity score: %f "% jc)
    jc_scores.append(jc)

    if i+1 == len(test_questions) or test_questions[i+1] !=  test_questions[i]:
      predictions_list.append(predicted_answers)

mean_em, mean_f1 = calculate_squad_em_f1(testing_path, 
                                         test_questions_ids, predictions_list)


print("Mean string comparison accuracy: {}".format(correct / questions_count))
print("Mean Cosine distance accuracy: {}".format(correct_cosine / questions_count))
print("Mean F1 Score: {}".format(np.mean(f1_scores)))
print("Mean EM Score: {}".format(np.mean(em_scores)))
print("Official SQuAD Mean F1 Score: {}".format(mean_f1))
print("Official SQuAD Mean EM Score: {}".format(mean_em))
print("Mean Bleu Score: {}".format(np.mean(bleu_scores)))
print("Mean Meteor Score: {}".format(np.mean(meteor_scores)))
print("Mean Jacard Similarity Score: {}".format(np.mean(jc_scores)))
print()

In [ ]:

answers = ["a pad of chloroform"]##each right answer is in a list
prediction = ['a pad of chloroform']
bleu_score = list_bleu(answers,prediction)
print(bleu_score)
print(answers)
print(prediction)

bleu_score = list_bleu(['captain daniels in the front seat'],['captain daniels in the front seat'])
print(bleu_score)

In [ ]:
##Use of meteor with example answers
##when there are multiple right answers for one question
answers = ["alternative answer of Q1"]##all right answers in one list
prediction = 'alternative answer of Q1' ## a string not a list
meteor_scores = meteor_score.meteor_score(answers,prediction)
print(meteor_scores)

answers = ["bed in the middle of a house on a lake"]##all right answers in one list
prediction = 'bed in the middle of a house on a lake' ## a string not a list
meteor_scores = meteor_score.meteor_score(answers,prediction)
print(meteor_scores)

In [ ]:
bleu_score = list_bleu(['small house on a hill a lake'],['small home on a hill a lake'])
print(bleu_score)

In [ ]:
actual = 'ethlychloride'
pred = 'ethlychloride'
compute_f1(actual, pred)

In [ ]:
actual = 'in a woollen shawl'
pred = 'woollen shawl'
get_jaccard_sim(actual, pred)